<a href="https://colab.research.google.com/github/ReeceRios/Intro2python/blob/main/Sentiment_Analysis_Hertie_Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sent Analysis
Source: https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis


In [29]:
!pip install transformers flair happytransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
'''
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
#autenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

#defining my worksheet
worksheet = gc.open('Data for Sentiment Analysis').sheet1
#get_all_values gives a list of rows
rows = worksheet.get_all_values()
#Convert to a DataFrame 
df = pd.DataFrame(rows)
#creating columns name
df.columns = df.iloc[0]
df = df.iloc[1:]
df.head()
 
'''

"\nfrom google.colab import auth\nimport gspread\nfrom google.auth import default\nimport pandas as pd\n#autenticating to google\nauth.authenticate_user()\ncreds, _ = default()\ngc = gspread.authorize(creds)\n\n#defining my worksheet\nworksheet = gc.open('Data for Sentiment Analysis').sheet1\n#get_all_values gives a list of rows\nrows = worksheet.get_all_values()\n#Convert to a DataFrame \ndf = pd.DataFrame(rows)\n#creating columns name\ndf.columns = df.iloc[0]\ndf = df.iloc[1:]\ndf.head()\n \n"

In [31]:
import pandas as pd 

df = pd.read_excel(r'https://github.com/ReeceRios/Intro2python/blob/main/Data%20for%20Sentiment%20Analysis.xlsx?raw=true')
print(df)

   Label                                            Comment
0    pos  Hi, thank you very much for the invitation. I ...
1    neg  Hi, so happy to hear that you are getting marr...
2    pos         Thanks for the invite, i am happy to come.
3    neg  I am very sorry, but i will not be able to att...
4    pos  Hello! Thanks your for your invitation! I am l...
5    neg  Hello my love! I am so sorry! I can not make i...
6    pos  Dear Anna, thank you so much for your invitati...
7    neg  Dear Anna and Pia, we were so happy to receive...
8    pos  Hi Anna, I was very happy to receive the invit...
9    neg  Hi Anna, thank you so much for the wedding inv...
10   pos  Thanks a lot for your invitation. We will be h...
11   neg  I appreciate your inivitation a lot. Unfortuna...
12   pos  Many thanks for the invitation. We are looking...
13   neg  Thanks a lot for your invitation. Unfortunatel...
14   pos  Dear Anna, good news! I can make it to your we...
15   neg  Dear Anna, unfortunately I can

In [32]:
import pandas as pd
from transformers import pipeline

classifier = pipeline("text-classification", model="finiteautomata/bertweet-base-sentiment-analysis", return_all_scores=True)
df['huggingface_prediction'] = df.Comment.apply(lambda txt: 'pos' if classifier(txt)[0][-1]['score'] >= 0.5 else 'neg')
df.head()


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/text_classification.py:92: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  UserWarning,


,Label,Comment,huggingface_prediction
0,pos,"Hi, thank you very much for the invitation. I ...",pos
1,neg,"Hi, so happy to hear that you are getting marr...",pos
2,pos,"Thanks for the invite, i am happy to come.",pos
3,neg,"I am very sorry, but i will not be able to att...",neg
4,pos,Hello! Thanks your for your invitation! I am l...,pos


In [33]:
#Flair 

import pandas as pd
import openpyxl
import flair
from flair.models import TextClassifier
from flair.data import Sentence
from segtok.segmenter import split_single


cellnumber = 0 #needed for while loop 

#creating pandas df from excel sheet with responses 

all_responses = df.copy()
totalrows = len(all_responses.index) - 1 #also needed for the while loop: will use to repeat for all rows in column

flair_preds = []
while cellnumber <= totalrows: 
    
    response = all_responses.Comment.iloc[cellnumber] #get response in new row
    classifier = TextClassifier.load('en-sentiment') #get flair text classifier for eng language 
    split_text = [sent for sent in split_single(response)] #split response into sentences 
    sentence = Sentence(split_text) #make a sentence for flair out of the split text
    
    classifier.predict(sentence) #sentiment analysis 
    labels = str(sentence.labels)
    print(sentence.labels) #print out result 
    
    if "NEGATIVE" in labels:
        flair_preds.append('neg')
    else: 
        flair_preds.append('pos')
   
    cellnumber = cellnumber + 1


2022-11-22 15:27:10,195 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt
['Sentence: "Hi, thank you very much for the invitation. I will definitely come to you wedding. I am very happy to spend the day with you!"'/'POSITIVE' (0.9923)]
2022-11-22 15:27:13,130 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt
['Sentence: "Hi, so happy to hear that you are getting married! Unfortunately I cannot make it to you wedding, because I am on vacation."'/'POSITIVE' (0.9688)]
2022-11-22 15:27:15,077 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt
['Sentence: "Thanks for the invite, i am happy to come."'/'POSITIVE' (0.989)]
2022-11-22 15:27:17,719 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt
['Sentence: "I am very sorry, but i will not be able to attend your wedding."'/'NEGATIVE' (0.9931)]
2022-11-22 15:27:20,905 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt
['Sentence: "Hello! Thanks your for your invi

In [34]:
df['flair_prediction'] = flair_preds
df

,Label,Comment,huggingface_prediction,flair_prediction
0,pos,"Hi, thank you very much for the invitation. I ...",pos,pos
1,neg,"Hi, so happy to hear that you are getting marr...",pos,pos
2,pos,"Thanks for the invite, i am happy to come.",pos,pos
3,neg,"I am very sorry, but i will not be able to att...",neg,neg
4,pos,Hello! Thanks your for your invitation! I am l...,pos,pos
5,neg,Hello my love! I am so sorry! I can not make i...,neg,neg
6,pos,"Dear Anna, thank you so much for your invitati...",pos,pos
7,neg,"Dear Anna and Pia, we were so happy to receive...",pos,pos
8,pos,"Hi Anna, I was very happy to receive the invit...",pos,pos
9,neg,"Hi Anna, thank you so much for the wedding inv...",pos,pos


In [35]:
from textblob import TextBlob


polarity_score = []

for i in range(0, df.shape[0]):
  score = TextBlob(df.Comment.iloc[i])
  score1 = score.sentiment[0]
  polarity_score.append(score1)

df['textblob_prediction'] = ['pos' if score > 0 else 'neg' for score in polarity_score]


In [36]:
from happytransformer import HappyTextClassification

happy_tc = HappyTextClassification(model_type="DISTILBERT", model_name = "distilbert-base-uncased-finetuned-sst-2-english", num_labels=2)

polarity_score_transform = []

for i in range(0, df.shape[0]):
  pred = happy_tc.classify_text(df.Comment.iloc[i]).label
  pred = 'pos' if pred == 'POSITIVE' else 'neg'
  polarity_score_transform.append(pred)

df['happytransformer_prediction'] = polarity_score_transform
df

,Label,Comment,huggingface_prediction,flair_prediction,textblob_prediction,happytransformer_prediction
0,pos,"Hi, thank you very much for the invitation. I ...",pos,pos,pos,pos
1,neg,"Hi, so happy to hear that you are getting marr...",pos,pos,pos,neg
2,pos,"Thanks for the invite, i am happy to come.",pos,pos,pos,pos
3,neg,"I am very sorry, but i will not be able to att...",neg,neg,neg,neg
4,pos,Hello! Thanks your for your invitation! I am l...,pos,pos,pos,pos
5,neg,Hello my love! I am so sorry! I can not make i...,neg,neg,neg,neg
6,pos,"Dear Anna, thank you so much for your invitati...",pos,pos,pos,pos
7,neg,"Dear Anna and Pia, we were so happy to receive...",pos,pos,pos,pos
8,pos,"Hi Anna, I was very happy to receive the invit...",pos,pos,pos,pos
9,neg,"Hi Anna, thank you so much for the wedding inv...",pos,pos,pos,pos


In [37]:
#Vader 

import nltk 
import pandas as pd
nltk.download("vader_lexicon") 


all_responses = df.copy()
totalrows = len(all_responses.index) - 1 #also needed for the while loop: will use to repeat for all rows in column

vader_preds = []
 
cellnumber = 0 #needed for while loop 

#creating pandas df from excel sheet with responses 


totalrows = len(all_responses.index) - 1 #also needed for the while loop: will use to repeat for all rows in column

while cellnumber <= totalrows: 
    
    from nltk.sentiment import SentimentIntensityAnalyzer as sia #needs to be redone every time or I get an error
    response = str(all_responses.iloc[cellnumber]) #get response in new row

#analyze response

    sia = sia() 
    score = sia.polarity_scores(response)
    
    if score["compound"] >= 0: 
        vader_preds.append("pos")
        
    elif score["compound"] < 0: 
        vader_preds.append("neg")
    
    cellnumber = cellnumber + 1


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [38]:
df['vader_prediction'] = vader_preds
df

,Label,Comment,huggingface_prediction,flair_prediction,textblob_prediction,happytransformer_prediction,vader_prediction
0,pos,"Hi, thank you very much for the invitation. I ...",pos,pos,pos,pos,pos
1,neg,"Hi, so happy to hear that you are getting marr...",pos,pos,pos,neg,pos
2,pos,"Thanks for the invite, i am happy to come.",pos,pos,pos,pos,pos
3,neg,"I am very sorry, but i will not be able to att...",neg,neg,neg,neg,neg
4,pos,Hello! Thanks your for your invitation! I am l...,pos,pos,pos,pos,pos
5,neg,Hello my love! I am so sorry! I can not make i...,neg,neg,neg,neg,pos
6,pos,"Dear Anna, thank you so much for your invitati...",pos,pos,pos,pos,pos
7,neg,"Dear Anna and Pia, we were so happy to receive...",pos,pos,pos,pos,pos
8,pos,"Hi Anna, I was very happy to receive the invit...",pos,pos,pos,pos,pos
9,neg,"Hi Anna, thank you so much for the wedding inv...",pos,pos,pos,pos,pos


In [39]:
from sklearn.metrics import f1_score, accuracy_score

In [40]:
models = ['huggingface_prediction',	'flair_prediction',	'textblob_prediction',	'happytransformer_prediction', 'vader_prediction']
rows = []
for model in models:
  row = {
      'model': model,
      'accuracy': accuracy_score(df.Label, df[model]),
      'f1_score': f1_score(df.Label, df[model], pos_label='pos'),
  }
  rows.append(row)

eval_df = pd.DataFrame(rows)
eval_df_sorted=eval_df.sort_values('f1_score', ascending=False)

In [41]:
eval_df_final = eval_df_sorted.style.highlight_max(subset = ['accuracy', 'f1_score'], color='lightgreen').highlight_min(subset = ['accuracy', 'f1_score'], color='pink')
eval_df_final

,model,accuracy,f1_score
3,happytransformer_prediction,0.800000,0.833333
0,huggingface_prediction,0.775000,0.816327
1,flair_prediction,0.750000,0.800000
2,textblob_prediction,0.650000,0.720000
4,vader_prediction,0.600000,0.714286


In [42]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 5.0 MB/s 


In [43]:
from xlsxwriter import Workbook

writer = pd.ExcelWriter('Final_Project_results.xlsx', engine='xlsxwriter')

df.to_excel(writer, sheet_name='Sentiment Analysis', index=False)
eval_df_final.to_excel(writer, sheet_name='Accuracy Scores', index=False)

writer.close()